In [1]:
from elasticsearch import Elasticsearch

In [2]:
es = Elasticsearch(
"https://localhost:9200",
basic_auth=("elastic","-_aPQVxoveqsQr40we7r"),
ca_certs="D:\ELASTIC SEARCH\elasticsearch-8.11.1\config\certs\http_ca.crt")
es.ping()

True

## Prepare the Data

In [3]:
import pandas as pd

In [4]:
df=pd.read_csv("myntra_products_catalog.csv").loc[:499]
df.head()

,ProductID,ProductName,ProductBrand,Gender,Price (INR),NumImages,Description,PrimaryColor
0,10017413,DKNY Unisex Black & Grey Printed Medium Trolle...,DKNY,Unisex,11745,7,"Black and grey printed medium trolley bag, sec...",Black
1,10016283,EthnoVogue Women Beige & Grey Made to Measure ...,EthnoVogue,Women,5810,7,Beige & Grey made to measure kurta with churid...,Beige
2,10009781,SPYKAR Women Pink Alexa Super Skinny Fit High-...,SPYKAR,Women,899,7,Pink coloured wash 5-pocket high-rise cropped ...,Pink
3,10015921,Raymond Men Blue Self-Design Single-Breasted B...,Raymond,Men,5599,5,Blue self-design bandhgala suitBlue self-desig...,Blue
4,10017833,Parx Men Brown & Off-White Slim Fit Printed Ca...,Parx,Men,759,5,"Brown and off-white printed casual shirt, has ...",White


In [5]:
df.isna().value_counts()

ProductID  ProductName  ProductBrand  Gender  Price (INR)  NumImages  Description  PrimaryColor
False      False        False         False   False        False      False        False           468
                                                                                   True             32
Name: count, dtype: int64

In [6]:
df.fillna("none", inplace=True)

In [7]:
df.isna().value_counts()

ProductID  ProductName  ProductBrand  Gender  Price (INR)  NumImages  Description  PrimaryColor
False      False        False         False   False        False      False        False           500
Name: count, dtype: int64

## Convert the relevant field into Vector using BERT model using sbert.net

In [9]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('all-mpnet-base-v2')

d:\SEMANTIC SEARCH\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [10]:
df["DescriptionVector"] = df["Description"].apply(lambda x: model.encode(x))
df.head()

,ProductID,ProductName,ProductBrand,Gender,Price (INR),NumImages,Description,PrimaryColor,DescriptionVector
0,10017413,DKNY Unisex Black & Grey Printed Medium Trolle...,DKNY,Unisex,11745,7,"Black and grey printed medium trolley bag, sec...",Black,"[0.027645882, -0.0026341544, -0.0035884061, 0...."
1,10016283,EthnoVogue Women Beige & Grey Made to Measure ...,EthnoVogue,Women,5810,7,Beige & Grey made to measure kurta with churid...,Beige,"[-0.02466069, -0.028755367, -0.02033252, 0.034..."
2,10009781,SPYKAR Women Pink Alexa Super Skinny Fit High-...,SPYKAR,Women,899,7,Pink coloured wash 5-pocket high-rise cropped ...,Pink,"[-0.04694324, 0.081827946, 0.048335165, -0.000..."
3,10015921,Raymond Men Blue Self-Design Single-Breasted B...,Raymond,Men,5599,5,Blue self-design bandhgala suitBlue self-desig...,Blue,"[-0.0150987515, -0.0102854, 0.009487293, -0.02..."
4,10017833,Parx Men Brown & Off-White Slim Fit Printed Ca...,Parx,Men,759,5,"Brown and off-white printed casual shirt, has ...",White,"[-0.01774658, 0.0062096184, 0.021813959, 0.026..."


In [11]:
es.ping()

True

## Create new index in Elasticsearch

In [12]:
from indexMapping import indexMapping

In [13]:
print(indexMapping)

{'properties': {'ProductID': {'type': 'long'}, 'ProductName': {'type': 'text'}, 'ProductBrand': {'type': 'text'}, 'Gender': {'type': 'text'}, 'Price (INR)': {'type': 'long'}, 'NumImages': {'type': 'long'}, 'Description': {'type': 'text'}, 'PrimaryColor': {'type': 'text'}, 'DescriptionVector': {'type': 'dense_vector', 'dims': 768, 'index': True, 'similarity': 'l2_norm'}}}


In [14]:
es.indices.create(index="all_products",mappings=indexMapping)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'all_products'})

## Ingest the data into index

In [15]:
record_list = df.to_dict("records")

In [16]:
record_list[0]

{'ProductID': 10017413,
 'ProductName': 'DKNY Unisex Black & Grey Printed Medium Trolley Bag',
 'ProductBrand': 'DKNY',
 'Gender': 'Unisex',
 'Price (INR)': 11745,
 'NumImages': 7,
 'Description': 'Black and grey printed medium trolley bag, secured with a TSA lockOne handle on the top and one on the side, has a trolley with a retractable handle on the top and four corner mounted inline skate wheelsOne main zip compartment, zip lining, two compression straps with click clasps, one zip compartment on the flap with three zip pocketsWarranty: 5 yearsWarranty provided by Brand Owner / Manufacturer',
 'PrimaryColor': ' Black',
 'DescriptionVector': array([ 2.76458822e-02, -2.63415440e-03, -3.58840614e-03,  5.13588078e-02,
         3.09661515e-02,  1.40507463e-02,  7.27061229e-03,  3.13872024e-02,
        -6.23786710e-02, -3.82887130e-03,  3.15214172e-02,  7.55471736e-02,
         2.12643947e-03,  4.64892797e-02,  5.07448763e-02, -1.71942171e-02,
         1.22892857e-02, -1.95682608e-02, -9.6

In [17]:
for record in record_list:
    try:
        es.index(index="all_products", document=record, id=record["ProductID"])
    except Exception as e:
        print(e)

In [18]:
es.count(index="all_products")

ObjectApiResponse({'count': 500, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}})

## Search the data 

In [19]:
input_keyword = "Blue shoes"
vector_of_input_keyword = model.encode(input_keyword)

In [21]:
query = {
    "field" : "DescriptionVector",
    "query_vector" : vector_of_input_keyword,
    "k" : 2,
    "num_candidates": 500,
}

res = es.knn_search(index="all_products", knn= query , source=["ProductName","Description"])
res["hits"]["hits"]

C:\Users\mugis\AppData\Local\Temp\ipykernel_16292\3138564689.py:8: ElasticsearchWarning: The kNN search API has been replaced by the `knn` option in the search API.
  res = es.knn_search(index="all_products", knn= query , source=["ProductName","Description"])


[{'_index': 'all_products',
  '_id': '10018013',
  '_score': 0.6142943,
  '_source': {'ProductName': 'Puma Men Blue Sneakers',
   'Description': 'A pair of round-toe blue sneakers, has regular styling, lace-up detailTextile upperCushioned footbedTextured and patterned outsoleWarranty: 3 monthsWarranty provided by brand/manufacturer'}},
 {'_index': 'all_products',
  '_id': '10018075',
  '_score': 0.6142943,
  '_source': {'ProductName': 'Puma Men Blue Sneakers',
   'Description': 'A pair of round-toe blue sneakers, has regular styling, lace-up detailTextile upperCushioned footbedTextured and patterned outsoleWarranty: 3 monthsWarranty provided by brand/manufacturer'}}]